In [2]:
import requests
from dotenv import load_dotenv
import os

# Replace with the full path to your .env file
load_dotenv(dotenv_path="/Users/ioa6870/repos/tbcrozier/diet-coach/.env")

api_key = os.getenv("USDA_API_KEY")
print("Loaded API key:", api_key)

def get_basic_nutrients(food_name, api_key):
    url = "https://api.nal.usda.gov/fdc/v1/foods/search"
    params = {
        "query": food_name,
        "pageSize": 1,
        "api_key": api_key
    }
    response = requests.get(url, params=params)
    data = response.json()

    # Get the first search result
    if not data.get("foods"):
        return f"No foods found for '{food_name}'"

    food = data["foods"][0]
    nutrients = {
        "calories": None,
        "protein_g": None,
        "carbs_g": None,
        "fat_g": None
    }

    for nutrient in food.get("foodNutrients", []):
        if nutrient["nutrientId"] == 1008:
            nutrients["calories"] = nutrient["value"]
        elif nutrient["nutrientId"] == 1003:
            nutrients["protein_g"] = nutrient["value"]
        elif nutrient["nutrientId"] == 1004:
            nutrients["fat_g"] = nutrient["value"]
        elif nutrient["nutrientId"] == 1005:
            nutrients["carbs_g"] = nutrient["value"]

    return {
        "description": food["description"],
        "fdcId": food["fdcId"],
        **nutrients
    }

# Example use:
print(get_basic_nutrients("potato", api_key))


Loaded API key: xhAZ31hvj9ubCwpHsSNXQxtXu1QVG6G4svvOh6eT
{'description': 'Potato patty', 'fdcId': 2709510, 'calories': 171, 'protein_g': 3.89, 'carbs_g': 13.49, 'fat_g': 11.32}


In [11]:
def estimate_meal_nutrition(meal_description, api_key):
    food_items = [item.strip() for item in meal_description.split(",")]

    total = {"calories": 0, "protein_g": 0, "carbs_g": 0, "fat_g": 0}
    details = []

    for food in food_items:
        result = get_basic_nutrients(food, api_key)
        if isinstance(result, dict):
            details.append(result)
            for key in total:
                total[key] += result.get(key, 0) or 0

    return {
        "meal_description": meal_description,
        "total_nutrition": total,
        "items": details
    }

meal = "3 eggs, toast with butter, a banana, coffee with cream"
print(estimate_meal_nutrition(meal, api_key))


{'meal_description': '3 eggs, toast with butter, a banana, coffee with cream', 'total_nutrition': {'calories': 986.0, 'protein_g': 29.65, 'carbs_g': 170.92, 'fat_g': 24.11}, 'items': [{'description': 'Eggs, Grade A, Large, egg white', 'fdcId': 747997, 'calories': 55.0, 'protein_g': 10.7, 'carbs_g': 2.36, 'fat_g': 0.0}, {'description': 'Melba toast', 'fdcId': 2707702, 'calories': 390, 'protein_g': 12.1, 'carbs_g': 76.6, 'fat_g': 3.2}, {'description': 'Bananas, dehydrated, or banana powder', 'fdcId': 173945, 'calories': 346, 'protein_g': 3.89, 'carbs_g': 88.3, 'fat_g': 1.81}, {'description': 'Cream, fluid, light (coffee cream or table cream)', 'fdcId': 170857, 'calories': 195, 'protein_g': 2.96, 'carbs_g': 3.66, 'fat_g': 19.1}]}
